## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용

In [44]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse # url encode
from tqdm import tqdm, tqdm_notebook

In [ ]:
driver = webdriver.Chrome()
url = 'https://search.danawa.com/dsearch.php?query=무선청소기&page=1&limit=120&sort=saveDESC&list=list'
driver.get(url)
time.sleep(5.0)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [ ]:
prodItems = soup.select('ul.product_list >li.prod_item')


In [ ]:
# id = "productItem19965230" 식으로 전개되므로 productItem 이 아니면 패스.
# 제품명 가져오기
prodItems[0].select('p.prod_name > a')[0].text.strip()

In [ ]:
# 제품스펙 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t','')

In [ ]:
# 제품가격 가져오기
# input 중 두번째 값에서 value값을 가져온다
prodItems[0].select('input')[1].get('value')

In [ ]:
# 제품 인트로
prodItems[0].select('p.intro_text')[0].text

##### 1번  페이지에서 상품 정보 추출

In [ ]:
prodData = []
for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip()
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t','')
        price = prodItem.select('input')[1].get('value')
        intro = prodItem.select('p.intro_text')[0].text
        prodData.append([prodName,specList,price])
    except:
        pass


In [ ]:
pd.DataFrame(prodData)

##### 여러페이지 크롤링 하는 방법


In [ ]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
            'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
            'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
            'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [ ]:
# 검색어에 공백이 있으면 URL에서 링크 클리이 안되는 문제 발생 --> urlEncode: parse.quote()를 통해서 문제 해결
getSearchPageUrl('페브리즈air',1)

In [ ]:
total_page = 10
for page in tqdm(range(1,total_page+1)):
    time.sleep(1)

In [ ]:
# 여러페이지 검색후 크롤링하는 작업
